<a href="https://colab.research.google.com/github/sourabhs701/Image-recommendation-system/blob/main/Fashion_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraies 

In [1]:
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import numpy as np
from numpy.linalg import norm
import os
from tqdm import tqdm
import pickle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Model Creation


In [3]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

94765736/94765736 [==============================] - 0s 0us/step


#Feature Extraction

In [4]:
def extract_features(img_path,model):
    img = image.load_img(img_path,target_size=(224,224))       #Loading Image
    img_array = image.img_to_array(img)                        #Converting image into np array
    expanded_img_array = np.expand_dims(img_array, axis=0)     #using expand.dims to Convert 3D array into 4D array
    preprocessed_img = preprocess_input(expanded_img_array)    #Preprocss_input
    result = model.predict(preprocessed_img).flatten()         #flatting
    normalized_result = result / norm(result)                  #Normalization

    return normalized_result

#Downloading kaggle Dataset

In [15]:
!pip install kaggle
from google.colab import files
files.upload()
#Make a directory named kaggle and copy the kaggle.json file there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# change the permission of the file
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d paramaggarwal/fashion-product-images-small
from zipfile import ZipFile
file_name = 'fashion-product-images-small.zip' #the file is your dataset exact name
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle (1).json
 99% 561M/565M [00:05<00:00, 137MB/s]
100% 565M/565M [00:05<00:00, 111MB/s]
Done


#Making list of path of images

In [16]:
filenames = []

for file in os.listdir('images'):
    filenames.append(os.path.join('images',file))

pickle.dump(filenames,open('filenames.pkl','wb'))

#Making list of features(2048) of images

In [ ]:
# feature_list = []

# for file in tqdm(filenames):    #using tqdm to know the current state of loop
#     feature_list.append(extract_features(file,model))

# pickle.dump(feature_list,open('Features.pkl','wb'))


#Testing

In [13]:
from sklearn.neighbors import NearestNeighbors
!pip install Streamlit
import streamlit as st

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 18.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 KB 11.5 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19581 sha256=a0f0dc017818dc7cafd9ddae6867f8511c1ac6c5e73ce4dfbfcc96cb765fe94b
  Stored in directory: /root/.cache/pip/wheels/19/09/72/3eb

In [10]:
fileName = pickle.load(open('/content/drive/MyDrive/filenames.pkl','rb'))
features = np.array(pickle.load(open('/content/drive/MyDrive/Features.pkl','rb')))

In [27]:
def recommend(features,feature_list):
    neighbors = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')
    neighbors.fit(feature_list)

    distances, index = neighbors.kneighbors([features])

    return index

def show(index):
  col1,col2,col3,col4,col5 = st.beta_columns(5)
  with col1:
    st.image(fileName[index[0][0]])
  with col2:
    st.image(fileName[index[0][1]])
  with col3:
    st.image(fileName[index[0][2]])
  with col4:
    st.image(fileName[index[0][3]])
  with col5:
    st.image(fileName[index[0][4]])
  
def save_file(uploaded_file):
  try:
    with open(os.path.join('Uploads',uploaded_file.name,'rb')) as f:
      f.write(uploaded_file.getbuffer())
      return 1
  except:
    return 0


st.title("Similar Clothes")
upload_file = st.file_uploader("Choose an image")
if upload_file is not None:
  save_file(upload_file)

sample_image = np.random.randint(0,len(fileName))
sample_path = fileName[sample_image]

st.image(sample_path)

sample_feature = extract_features(sample_path,model)
index = recommend(sample_feature , features)

show(index)

for i in index[0][1:6]:
  print(fileName[i])

1/1 [==============================] - 0s 208ms/step
images/27083.jpg
images/47576.jpg
images/46622.jpg
images/45416.jpg
images/44461.jpg
